In [10]:
import csv
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import svm
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
import numpy as np

In [11]:
dataset = pd.read_csv('../../../data/dataset_small.csv')

print(dataset.columns)
print(dataset.shape)

Index(['qty_dot_url', 'qty_hyphen_url', 'qty_underline_url', 'qty_slash_url',
       'qty_questionmark_url', 'qty_equal_url', 'qty_at_url', 'qty_and_url',
       'qty_exclamation_url', 'qty_space_url',
       ...
       'qty_ip_resolved', 'qty_nameservers', 'qty_mx_servers', 'ttl_hostname',
       'tls_ssl_certificate', 'qty_redirects', 'url_google_index',
       'domain_google_index', 'url_shortened', 'phishing'],
      dtype='object', length=112)
(58645, 112)


In [12]:
train_data, test_data, train_labels, test_labels = train_test_split(
    dataset.iloc[:, 0:97], 
    dataset.iloc[:, -1], 
    test_size=0.2
)

In [13]:
dt_clf = tree.DecisionTreeClassifier()
knn_clf = KNeighborsClassifier(n_neighbors=50)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)

In [14]:
from sklearn.ensemble import VotingClassifier

estimators=[('knn', knn_clf), ('rf', rf_clf), ('dt', dt_clf)]
ensemble = VotingClassifier(estimators, voting='hard')
ensemble.fit(train_data, train_labels)

VotingClassifier(estimators=[('knn', KNeighborsClassifier(n_neighbors=50)),
                             ('rf', RandomForestClassifier(random_state=0)),
                             ('dt', DecisionTreeClassifier())])

In [15]:
pred_labels_val = cross_val_predict(ensemble, train_data, train_labels, cv=10)
conf_mat = confusion_matrix(train_labels, pred_labels_val)
print(conf_mat)

[[20495  1909]
 [ 2536 21976]]


In [16]:
with open('../small-ensemble-knn-rf-dt.pkl','wb') as f:
     pickle.dump(ensemble,f)

In [17]:
pred_labels_test = ensemble.predict(test_data)
print("Accuracy:", metrics.accuracy_score(test_labels, pred_labels_test))      # Calculating the accuracy of the model
print("Precision:", metrics.precision_score(test_labels, pred_labels_test))    # Calculating the precision of the model
print("Recall:", metrics.recall_score(test_labels, pred_labels_test)) 

Accuracy: 0.9068121749509762
Precision: 0.9269986449864499
Recall: 0.8920945395273023
